In [3]:
# Cell 1: Set up the path
import sys
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()
# Get the SRC_PATH from the environment variable
src_path = os.getenv('SRC_PATH')

print(f"SRC_PATH: {src_path}")

if src_path:
    # Add the src path to sys.path
    sys.path.append(os.path.abspath(src_path))

SRC_PATH: D:\CODE\de_projects\finance_hub\src


In [4]:
from pipeline.commons.constants import VnstockDataSources
from pipeline.commons.connectors.vnstock_lib import VnstockLibConnector
from pipeline.extract.vnstock_lib import VnstockLibExtractor

D:\CODE\de_projects\finance_hub\src is already in sys.path
D:\CODE\de_projects\finance_hub\src is already in sys.path


## Connector

In [5]:
connector = VnstockLibConnector(init_symbol='MBB')

2024-10-27 12:00:09,252 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Connecting to VnstockLib: Symbol: MBB, Source: VCI
Connected to VnstockLib.


In [6]:
connector.list_all_symbols()

,ticker,organ_name
0,A32,Công ty Cổ phần 32
1,AAA,Công ty Cổ phần Nhựa An Phát Xanh
2,AAM,Công ty Cổ Phần Thủy Sản MeKong
3,AAS,Công ty Cổ phần Chứng khoán SmartInvest
4,AAT,Công ty Cổ phần Tập đoàn Tiên Sơn Thanh Hóa
...,...,...
1593,XPH,Công ty Cổ phần Xà phòng Hà Nội
1594,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái
1595,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái
1596,YEG,Công ty Cổ phần Tập đoàn Yeah1


In [7]:
full_info = connector.get_full_company_info()

Getting full company info


In [8]:
# connector.list_all_symbols()

In [9]:
# connector.get_symbols_by_exchange()

In [10]:
# connector.get_full_company_info()

In [11]:
connector.get_stock_quote_history_df()

2024-10-27 12:00:11,335 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Fetching history for symbol: MBB


,time,open,high,low,close,volume,symbol
0,2011-11-01,2.55,2.55,2.42,2.42,2719290,MBB
1,2011-11-02,2.39,2.42,2.32,2.32,1255220,MBB
2,2011-11-03,2.28,2.32,2.21,2.30,1193910,MBB
3,2011-11-04,2.32,2.32,2.27,2.27,483860,MBB
4,2011-11-07,2.27,2.27,2.16,2.16,1158750,MBB
...,...,...,...,...,...,...,...
3238,2024-10-21,25.75,25.90,25.55,25.55,9410500,MBB
3239,2024-10-22,25.50,25.65,25.15,25.25,20327200,MBB
3240,2024-10-23,25.25,25.35,25.15,25.25,11634500,MBB
3241,2024-10-24,25.25,25.40,24.70,24.70,16246700,MBB


In [12]:
connector.get_stock_quote_history_dict()

2024-10-27 12:00:12,066 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


{'MBB': [{'time': 1320105600000,
   'open': 2.55,
   'high': 2.55,
   'low': 2.42,
   'close': 2.42,
   'volume': 2719290},
  {'time': 1320192000000,
   'open': 2.39,
   'high': 2.42,
   'low': 2.32,
   'close': 2.32,
   'volume': 1255220},
  {'time': 1320278400000,
   'open': 2.28,
   'high': 2.32,
   'low': 2.21,
   'close': 2.3,
   'volume': 1193910},
  {'time': 1320364800000,
   'open': 2.32,
   'high': 2.32,
   'low': 2.27,
   'close': 2.27,
   'volume': 483860},
  {'time': 1320624000000,
   'open': 2.27,
   'high': 2.27,
   'low': 2.16,
   'close': 2.16,
   'volume': 1158750},
  {'time': 1320710400000,
   'open': 2.12,
   'high': 2.14,
   'low': 2.07,
   'close': 2.11,
   'volume': 1105490},
  {'time': 1320796800000,
   'open': 2.09,
   'high': 2.11,
   'low': 2.04,
   'close': 2.04,
   'volume': 740500},
  {'time': 1320883200000,
   'open': 2.0,
   'high': 2.12,
   'low': 2.0,
   'close': 2.07,
   'volume': 1258020},
  {'time': 1320969600000,
   'open': 2.11,
   'high': 2.14,
  

## Extractor

In [13]:
import time

In [14]:
extractor = VnstockLibExtractor()

2024-10-27 12:00:12,741 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Connecting to VnstockLib: Symbol: VNN, Source: VCI
Connected to VnstockLib.


In [19]:
df_all = extractor.get_all_stock_quote_histories_df(symbols=['MBB', 'VCB', 'VIC', 'VNM'])

2024-10-27 12:03:45,679 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Processing batch 1: ['MBB', 'VCB', 'VIC', 'VNM']
Fetching history for symbol: MBB


2024-10-27 12:03:46,146 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Fetching history for symbol: VCB


2024-10-27 12:03:46,587 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Fetching history for symbol: VIC


2024-10-27 12:03:47,063 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Fetching history for symbol: VNM


In [22]:
df_all.to_json(f'{src_path}/test_data_gitignore/stock_quote_histories_df.json', orient='records')

In [25]:
dict_all = extractor.get_all_stock_quote_histories_dict(symbols=['MBB', 'VCB', 'VIC', 'VNM'])


import json
# Write the dictionary to a JSON file
try:
    with open(f'{src_path}/test_data_gitignore/stock_quote_histories_dict.json', 'w') as json_file:
        json.dump(dict_all, json_file, indent=4)  # Use indent for pretty-printing
    print(f"Data successfully written to {dict_all}")
except Exception as e:
    print(f"Error writing to JSON file: {e}")

2024-10-27 12:08:43,862 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Processing batch 1: ['MBB', 'VCB', 'VIC', 'VNM']
Fetching history for symbol: MBB


2024-10-27 12:08:44,370 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Fetching history for symbol: VCB


2024-10-27 12:08:44,885 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Fetching history for symbol: VIC


2024-10-27 12:08:45,369 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Fetching history for symbol: VNM
Error writing to JSON file: Object of type Timestamp is not JSON serializable


Error writing to JSON file: name 'dict_all' is not defined


In [17]:
len(extractor.get_symbols())

1598

## Transformer

## Loader